# Building Machine Learning Classifiers: Evaluate Gradient Boosting with GridSearchCV

**Grid-search:** Exhaustively search all parameter combinations in a given grid to determine the best model.

**Cross-validation:** Divide a dataset into k subsets and repeat the holdout method k times where a different subset is used as the holdout set in each iteration.

### Read in text

In [1]:
import nltk
import pandas as pd
import re
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
import string

stopwords = nltk.corpus.stopwords.words('english')
ps = nltk.PorterStemmer()

data = pd.read_csv("SMSSpamCollection.tsv", sep='\t')
data.columns = ['label', 'body_text']

def count_punct(text):
    count = sum([1 for char in text if char in string.punctuation])
    return round(count/(len(text) - text.count(" ")), 3)*100

data['body_len'] = data['body_text'].apply(lambda x: len(x) - x.count(" "))
data['punct%'] = data['body_text'].apply(lambda x: count_punct(x))

def clean_text(text):
    text = "".join([word.lower() for word in text if word not in string.punctuation])
    tokens = re.split('\W+', text)
    text = [ps.stem(word) for word in tokens if word not in stopwords]
    return text

# TF-IDF
tfidf_vect = TfidfVectorizer(analyzer=clean_text)
X_tfidf = tfidf_vect.fit_transform(data['body_text'])
X_tfidf_feat = pd.concat([data['body_len'], data['punct%'], pd.DataFrame(X_tfidf.toarray())], axis=1)

# CountVectorizer
count_vect = CountVectorizer(analyzer=clean_text)
X_count = count_vect.fit_transform(data['body_text'])
X_count_feat = pd.concat([data['body_len'], data['punct%'], pd.DataFrame(X_count.toarray())], axis=1)

X_count_feat.head()

,body_len,punct%,0,1,2,3,4,5,6,7,...,8094,8095,8096,8097,8098,8099,8100,8101,8102,8103
0,128,4.7,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,49,4.1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,62,3.2,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,28,7.1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,135,4.4,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### Exploring parameter settings using GridSearchCV

In [2]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import GridSearchCV

In [3]:
gb = GradientBoostingClassifier()
param = {
    'n_estimators': [100, 150], 
    'max_depth': [7, 11, 15],
    'learning_rate': [0.1]
}

clf = GridSearchCV(gb, param, cv=5, n_jobs=-1)
cv_fit = clf.fit(X_tfidf_feat, data['label'])
pd.DataFrame(cv_fit.cv_results_).sort_values('mean_test_score', ascending=False)[0:5]

/Users/nirbhayworlikar/opt/anaconda3/envs/Practice/lib/python3.8/site-packages/sklearn/utils/validation.py:1858: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  warnings.warn(
/Users/nirbhayworlikar/opt/anaconda3/envs/Practice/lib/python3.8/site-packages/sklearn/utils/validation.py:1858: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  warnings.warn(
/Users/nirbhayworlikar/opt/anaconda3/envs/Practice/lib/python3.8/site-packages/sklearn/utils/validation.py:1858: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  warnings.warn(
/Users/nirbhayworlikar/opt/anaconda3/envs/Practice/lib/python3.8/site-packages/sklearn/utils/validation.py:1858: FutureWarning: Feature names only support names th

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_learning_rate,param_max_depth,param_n_estimators,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
5,478.580080,5.183348,0.457206,0.081451,0.1,15,150,"{'learning_rate': 0.1, 'max_depth': 15, 'n_est...",0.965889,0.976661,0.970350,0.968553,0.970350,0.970361,0.003548,1
3,436.932391,9.218394,0.740734,0.077866,0.1,11,150,"{'learning_rate': 0.1, 'max_depth': 11, 'n_est...",0.965889,0.978456,0.969452,0.969452,0.965858,0.969821,0.004604,2
4,360.346711,7.792814,0.730447,0.137494,0.1,15,100,"{'learning_rate': 0.1, 'max_depth': 15, 'n_est...",0.964991,0.976661,0.967655,0.970350,0.968553,0.969642,0.003912,3
1,259.329088,5.423151,0.768586,0.228408,0.1,7,150,"{'learning_rate': 0.1, 'max_depth': 7, 'n_esti...",0.964991,0.979354,0.971249,0.964061,0.966757,0.969282,0.005610,4
2,255.275828,0.346902,0.649934,0.061630,0.1,11,100,"{'learning_rate': 0.1, 'max_depth': 11, 'n_est...",0.965889,0.979354,0.967655,0.967655,0.965858,0.969282,0.005098,5


In [4]:
gb = GradientBoostingClassifier()
param = {
    'n_estimators': [50, 100, 150], 
    'max_depth': [7, 11, 15],
    'learning_rate': [0.1]
}

clf = GridSearchCV(gb, param, cv=5, n_jobs=-1)
cv_fit = clf.fit(X_count_feat, data['label'])
pd.DataFrame(cv_fit.cv_results_).sort_values('mean_test_score', ascending=False)[0:5]

/Users/nirbhayworlikar/opt/anaconda3/envs/Practice/lib/python3.8/site-packages/sklearn/utils/validation.py:1858: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  warnings.warn(
/Users/nirbhayworlikar/opt/anaconda3/envs/Practice/lib/python3.8/site-packages/sklearn/utils/validation.py:1858: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  warnings.warn(
/Users/nirbhayworlikar/opt/anaconda3/envs/Practice/lib/python3.8/site-packages/sklearn/utils/validation.py:1858: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  warnings.warn(
/Users/nirbhayworlikar/opt/anaconda3/envs/Practice/lib/python3.8/site-packages/sklearn/utils/validation.py:1858: FutureWarning: Feature names only support names th

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_learning_rate,param_max_depth,param_n_estimators,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
5,409.545871,14.250952,0.817925,0.183267,0.1,11,150,"{'learning_rate': 0.1, 'max_depth': 11, 'n_est...",0.964093,0.977558,0.968553,0.965858,0.969452,0.969103,0.004638,1
2,272.024647,5.880425,0.673397,0.119295,0.1,7,150,"{'learning_rate': 0.1, 'max_depth': 7, 'n_esti...",0.964991,0.978456,0.970350,0.964061,0.967655,0.969103,0.005166,2
8,382.801950,52.678629,0.349156,0.043291,0.1,15,150,"{'learning_rate': 0.1, 'max_depth': 15, 'n_est...",0.963196,0.976661,0.967655,0.964960,0.971249,0.968744,0.004801,3
4,251.678625,1.305067,0.767970,0.198974,0.1,11,100,"{'learning_rate': 0.1, 'max_depth': 11, 'n_est...",0.964991,0.977558,0.965858,0.963163,0.969452,0.968204,0.005105,4
1,177.080146,2.443042,0.652412,0.061074,0.1,7,100,"{'learning_rate': 0.1, 'max_depth': 7, 'n_esti...",0.964093,0.978456,0.966757,0.962264,0.967655,0.967845,0.005640,5
